In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd

In [51]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

In [23]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})

In [4]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [3]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)

In [52]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

In [55]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

In [56]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "data_hora" : now.strftime("%d/%m/%Y %H:%M:%S")
    }
    produtos.append(produto)


In [57]:
df = pd.DataFrame(produtos)
df

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,data_hora
0,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"1.399,89","1.329,90",02/09/2020 20:18:02
1,Asus,Placa de Vídeo Asus TUF NVIDIA GeForce RTX 206...,"2.894,00","2.658,71","2.259,90",02/09/2020 20:18:02
2,Asus,Placa de Vídeo Asus TUF3 NVIDIA GeForce GTX 16...,0,"2.420,95","2.299,90",02/09/2020 20:18:02
3,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"1.578,84","1.499,90",02/09/2020 20:18:02
4,Gigabyte,Placa de Vídeo Gigabyte GTX 1660 Super OC NVID...,0,"2.210,42","2.099,90",02/09/2020 20:18:02
...,...,...,...,...,...,...
370,Pny,"Placa de Vídeo PNY NVIDIA Quadro P400, 2GB, GD...",0,"1.311,65","1.114,90",02/09/2020 20:18:02
371,Powercolor,Placa de Vídeo PowerColor Red Dragon AMD Radeo...,0,"783,41","665,90",02/09/2020 20:18:02
372,MSI,Placa de Vídeo MSI NVIDIA GeForce GTX 1650 Gam...,"1.599,88","1.388,12","1.179,90",02/09/2020 20:18:02
373,ASRock,Placa de Vídeo ASRock AMD Radeon RX 5700 XT Ph...,0,"3.835,18","3.259,90",02/09/2020 20:18:02


In [34]:
# containers